# Data Preprocessing, Averaging over all csv-files and Saving the averaged data into new csv-file

In [239]:
import os
import pandas as pd
import json 
from collections import Counter

# extract the code into a function called preprocess_data so that we can use it later for new data
def preprocess_data(averaged_df, directory, file):
    df = pd.read_csv(directory + file, sep=';')
    
    # Drop unnecessary and outdated columns
    deleted_columns = ['timeStampNetwork', 'timeStampGPS', 'locationDescription', 'people', 'latitudeGPS', 'longitudeGPS', 'latitudeNetwork', 'longitudeNetwork', 'minCn0GPS', 'maxCn0GPS', 'meanCn0GPS', 'minCn0Bluetooth', 'maxCn0Bluetooth', 'minCn0Wifi', 'maxCn0Wifi', 'meanCn0Wifi', 'bAccuracyNetwork', 'speedAccuracyNetwork', 'cellType', 'networkLocationType', 'hAccuracyNetwork', 'vAccuracyNetwork', 'speedAccuracyNetwork', 'bAccuracyNetwork', 'nrBlDevices', 'hAccuracyGPS', 'minCn0Bl', 'meanCn0Bl','maxCn0Bl', 'bAccuracyGPS', 'speedAccuracyGPS', 'vAccuracyGPS', 'nrWifiDevices']
    df = safe_delete(df, deleted_columns)
    
    # Remove first x rows and reset begin index to 0
    removedRows = 8
    df = df.iloc[removedRows:]
    df = df.reset_index(drop=True)
    
    # Load satellites json
    df['satellites'] = df['satellites'].apply(lambda x: json.loads(x))
    
    # Add cn0 column for easier computation of statistics
    df['satellite_cn0'] = df['satellites'].apply(lambda x: [sat['cn0'] for sat in x])
    
    # Calculate min, max, mean. median, mode, variance, standard deviation and range of the satellite cn0
    df['satellite_cn0_min'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).min() if not pd.Series(x).empty else 0)
    df['satellite_cn0_max'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).max() if not pd.Series(x).empty else 0)
    df['satellite_cn0_mean'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).mean() if not pd.Series(x).empty else 0)    
    df['satellite_cn0_median'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).median() if not pd.Series(x).empty else 0)
    df['satellite_cn0_mode'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).mode()[0] if not pd.Series(x).mode().empty else 0)
    df['satellite_cn0_std'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).std() if not pd.Series(x).empty else 0)
    df['satellite_cn0_range'] = df['satellite_cn0'].apply(lambda x: pd.Series(x).max() - pd.Series(x).min() if not pd.Series(x).empty else 0)
    
    # load the bluetooth json and load rssi into a new column
    df['bluetoothDevices'] = df['bluetoothDevices'].apply(lambda x: json.loads(x))
    df['bluetooth_rssi'] = df['bluetoothDevices'].apply(lambda x: [device['rssi'] for device in x])
    
     # Calculate statistical figures for the bluetooth devices
    df['bluetooth_rssi_min'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).min() if not pd.Series(x).empty else 0)
    df['bluetooth_rssi_max'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).max() if not pd.Series(x).empty else 0)
    df['bluetooth_rssi_mean'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).mean() if not pd.Series(x).empty else 0)
    df['bluetooth_rssi_median'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).median() if not pd.Series(x).empty else 0)
    df['bluetooth_rssi_mode'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).mode()[0] if not pd.Series(x).mode().empty else 0)
    df['bluetooth_rssi_std'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).std() if not pd.Series(x).empty else 0)
    df['bluetooth_rssi_range'] = df['bluetooth_rssi'].apply(lambda x: pd.Series(x).max() - pd.Series(x).min() if not pd.Series(x).empty else 0)
    
    # load the wifi json and load rssi into a new column
    df['wifiDevices'] = df['wifiDevices'].apply(lambda x: json.loads(x))
    df['wifi_rssi'] = df['wifiDevices'].apply(lambda x: [device['level'] for device in x])
    
     # Calculate statistical figures for the wifi devices
    df['wifi_rssi_min'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).min() if not pd.Series(x).empty else 0)
    df['wifi_rssi_max'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).max() if not pd.Series(x).empty else 0)
    df['wifi_rssi_mean'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).mean() if not pd.Series(x).empty else 0)
    df['wifi_rssi_median'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).median() if not pd.Series(x).empty else 0)
    df['wifi_rssi_mode'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).mode()[0] if not pd.Series(x).mode().empty else 0)
    df['wifi_rssi_range'] = df['wifi_rssi'].apply(lambda x: pd.Series(x).max() - pd.Series(x).min() if not pd.Series(x).empty else 0)
    
    # Drop list columns
    df.drop(columns=['satellites', 'bluetoothDevices', 'wifiDevices', 'satellite_cn0', 'bluetooth_rssi', 'wifi_rssi'], inplace=True)
    
     # Average over all columns for numeric values and take the first of non-numeric to have a single row
    df_label = df.iloc[0]['label']
    
    df.drop(columns=['label'], inplace=True)
    
    df = df.mean().to_frame().T
    
    df['label'] = df_label
    
    #place label at the beginning
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    averaged_df = pd.concat([averaged_df, df], ignore_index=True)
    return averaged_df

def safe_delete(df, columns):
    for column in columns:
        if column in df.columns:
            df = df.drop(columns=[column])
    return df

files_directory = '../Daten/firsttry/'
averaged_path = '../Daten/averaged_data.csv'

if os.path.exists(averaged_path):
    os.remove(averaged_path)
    print(f"Removed {averaged_path}")

# Get list of CSV files
csv_files = [file for file in os.listdir(files_directory) if file.endswith('.csv')]
print(f'Found {len(csv_files)} CSV files')

# Get nr of files, where name starts with indoor and outdoor
indoor_files = [file for file in csv_files if file.startswith('Indoor')]
outdoor_files = [file for file in csv_files if file.startswith('Outdoor')]
print(f'Found {len(indoor_files)} indoor files and {len(outdoor_files)} outdoor files')

averaged_data = pd.DataFrame()

for file in csv_files:
    averaged_data = preprocess_data(averaged_data, files_directory, file)

averaged_data.sort_values(by=['label'], inplace=True)
averaged_data

Found 181 CSV files
Found 101 indoor files and 80 outdoor files


,label,cellStrength,nrSatellitesInView,nrSatellitesInFix,satellite_cn0_min,satellite_cn0_max,satellite_cn0_mean,satellite_cn0_median,satellite_cn0_mode,satellite_cn0_std,...,bluetooth_rssi_median,bluetooth_rssi_mode,bluetooth_rssi_std,bluetooth_rssi_range,wifi_rssi_min,wifi_rssi_max,wifi_rssi_mean,wifi_rssi_median,wifi_rssi_mode,wifi_rssi_range
0,Indoor,3.000000,44.526316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-92.315789,-98.157895,23.176776,57.157895,-88.947368,-47.210526,-71.832468,-73.210526,-88.947368,41.736842
124,Indoor,3.900000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-86.425000,-91.250000,12.659495,54.000000,-91.000000,-75.000000,-81.315789,-81.000000,-82.000000,16.000000
123,Indoor,3.850000,44.350000,5.150000,3.910000,4.925000,4.322429,4.197500,3.910000,0.404397,...,-89.350000,-90.600000,9.315417,29.050000,-90.000000,-70.000000,-80.428571,-82.500000,-86.000000,20.000000
122,Indoor,3.461538,47.961538,14.615385,16.561538,32.180769,23.561719,23.036538,18.980769,4.684867,...,-87.153846,-96.538462,12.494974,49.000000,-87.076923,-65.038462,-76.438292,-75.076923,-85.538462,22.038462
120,Indoor,4.000000,41.464286,21.928571,14.975000,39.342857,25.829818,24.850000,19.964286,7.368629,...,-77.357143,-86.000000,11.821742,39.285714,-86.000000,-66.000000,-76.000000,-76.000000,-86.000000,20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Outdoor,4.000000,51.000000,30.666667,20.552381,45.809524,33.788607,33.992857,32.604762,6.129338,...,-91.500000,-93.000000,16.766700,57.000000,-94.190476,-73.952381,-86.246753,-87.142857,-93.428571,20.238095
101,Outdoor,3.000000,55.782609,24.000000,18.056522,41.169565,30.775447,31.476087,24.960870,7.085132,...,-95.434783,-96.913043,11.094607,36.652174,-89.695652,-80.086957,-85.326087,-85.760870,-89.478261,9.608696
49,Outdoor,4.000000,48.000000,28.166667,15.827778,42.005556,26.505713,26.033333,22.177778,6.582545,...,-94.055556,-98.111111,21.116507,51.111111,-93.777778,-79.333333,-86.519048,-86.444444,-90.888889,14.444444
114,Outdoor,3.000000,44.000000,23.388889,15.300000,46.344444,30.747029,29.783333,22.972222,9.213696,...,-93.305556,-97.944444,12.099497,49.000000,-93.666667,-64.666667,-81.062309,-82.472222,-84.722222,29.000000


# Train the Random Forest Classifier

## Randomize and Split the Data for Random Forest Classifier

In [240]:
from sklearn.model_selection import train_test_split

# Randomize the data
averaged_data = averaged_data.sample(frac=1).reset_index(drop=True)

# drop the label column
X = averaged_data.drop(columns=['label'], axis=1)
Y = averaged_data['label']

# Split the data into training and testing data
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.0001)
X_train = X
Y_train = Y

print(f'Training data shape: {X_train.shape}')
#print(f'Testing data shape: {X_test.shape}')

Training data shape: (181, 23)


## Train a Random Forest Classifier

In [241]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train the Random Forest Classifier with regularization
clf = RandomForestClassifier()

# Train the classifier
clf.fit(X_train, Y_train)

# Cross-validation
scores = cross_val_score(clf, X_train, Y_train, cv=10)
print(f"Cross-validation scores: {scores}")

# Predict the labels of the test data
#y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
#accuracy = accuracy_score(Y_test, y_pred)
#print(f'Accuracy: {accuracy}')

Cross-validation scores: [0.94736842 1.         0.94444444 0.94444444 1.         0.83333333
 1.         0.88888889 0.94444444 0.83333333]


## Get the Feature Importances

In [242]:
# Print feature importances of the selected features
feature_importances = clf.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

,feature,importance
5,satellite_cn0_mean,0.204191
7,satellite_cn0_mode,0.143392
4,satellite_cn0_max,0.129275
2,nrSatellitesInFix,0.108345
6,satellite_cn0_median,0.106004
9,satellite_cn0_range,0.089437
3,satellite_cn0_min,0.042918
8,satellite_cn0_std,0.040546
1,nrSatellitesInView,0.018440
12,bluetooth_rssi_mean,0.017509


## Save the model and tryout the model with new data

In [243]:
import joblib

# Save the model
model_path = '../Daten/random_forest_classifier.joblib'
joblib.dump(clf, model_path)
print(f'Saved model to {model_path}')

Saved model to ../Daten/random_forest_classifier.joblib


# Test the model with new data in validation folder in this notebook

In [244]:
validation_files_directory = '../Daten/validation/'
validation_averaged_path = '../Daten/validation_averaged_data.csv'
model_path = '../Daten/random_forest_classifier.joblib'

# Load the model
model = joblib.load(model_path)

validation_averaged_data = pd.DataFrame()
validation_description = pd.DataFrame()
validation_files = [file for file in os.listdir(validation_files_directory) if file.endswith('.csv')]

for file in validation_files:
    df = pd.read_csv(validation_files_directory + file, sep=';')
    locationDescription = df['locationDescription'][0]
    locationPeople = df['people'][0]
    
    validation_description = pd.concat([validation_description, pd.DataFrame({'locationDescription': [locationDescription], 'people': [locationPeople], 'file': [file]})], ignore_index=True)

    validation_averaged_data = preprocess_data(validation_averaged_data, validation_files_directory, file)

# Predict the labels of the validation data
validation_predictions = model.predict(validation_averaged_data.drop(columns=['label']))
validation_probabilities = model.predict_proba(validation_averaged_data.drop(columns=['label']))

#print label and the corresponding prediction
for label, description, people, prediction, probability, file in zip(validation_averaged_data['label'], validation_description['locationDescription'], validation_description['people'], validation_predictions, validation_probabilities, validation_description['file']):
    if label != prediction:
        print("--------------------")
        print(f'Label: {label}, Prediction: {prediction}, Description: {description}, People: {people},  Probability: {probability}, File: {file}')
   

--------------------
Label: Indoor, Prediction: Outdoor, Description: Nähe Fenster, People: viel,  Probability: [0.47 0.53], File: Indoor_2024-03-29 13_41_40.csv


## Setup LSTM Model

In [245]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the LSTM model
model = Sequential()

model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

#convert all columns to float except the label column
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train = X_train.astype(float)
Y_train = Y_train.map({'Indoor': 0, 'Outdoor': 1})
Y_train = Y_train.astype(float)

X_test = X_test.astype(float)
Y_test = Y_test.map({'Indoor': 0, 'Outdoor': 1})
Y_train = Y_train.astype(float)

#print accuracy of the model with the averaged data
X_train = np.array(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test), batch_size=32)

print(f'Accuracy: {model.evaluate(X_test, Y_test)[1]}')



Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_75 (LSTM)              (None, 23, 128)           66560     
                                                                 
 dropout_100 (Dropout)       (None, 23, 128)           0         
                                                                 
 lstm_76 (LSTM)              (None, 23, 128)           131584    
                                                                 
 dropout_101 (Dropout)       (None, 23, 128)           0         
                                                                 
 lstm_77 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_102 (Dropout)       (None, 128)               0         
                                                                 
 dense_50 (Dense)            (None, 32)              

## Validate the LSTM Model

In [246]:
#map the label column to 0 and 1
validation_averaged_data['label'] = validation_averaged_data['label'].map({'Indoor': 0, 'Outdoor': 1})

#convert all columns to float except the label column
validation_averaged_data = validation_averaged_data.astype(float)

X_validation = validation_averaged_data.drop(columns=['label'])
Y_validation = validation_averaged_data['label']

X_validation = np.array(X_validation)
X_validation = np.reshape(X_validation, (X_validation.shape[0], X_validation.shape[1], 1))

Y_validation = Y_validation.astype(float)

#predict the labels of the validation data for each data point
validation_predictions = model.predict(X_validation)

#print label and the corresponding prediction
for label, description, people, prediction, file in zip(validation_averaged_data['label'], validation_description['locationDescription'], validation_description['people'], validation_predictions, validation_description['file']):
        print(f'Label: {label}, Prediction: {prediction}, Description: {description}, People: {people},  File: {file}')

1/1 [==============================] - 0s 347ms/step
Label: 1.0, Prediction: [0.033414   0.96658593], Description: Überdacht, People: viel,  File: Outdoor_2024-03-30 20_23_34.csv
Label: 0.0, Prediction: [0.99899    0.00101003], Description: Nähe Fenster, People: weniger als 5,  File: Indoor_2024-04-01 16_33_22.csv
Label: 1.0, Prediction: [0.02514686 0.9748531 ], Description: Häuserschlucht, People: viel,  File: Outdoor_2024-03-29 13_46_20.csv
Label: 0.0, Prediction: [0.99767894 0.00232114], Description: Raummitte, People: viel,  File: Indoor_2024-03-29 13_43_31.csv
Label: 0.0, Prediction: [0.9989717 0.0010283], Description: Raummitte, People: weniger als 5,  File: Indoor_2024-03-29 13_05_59.csv
Label: 1.0, Prediction: [0.01867104 0.98132896], Description: Häuserschlucht, People: weniger als 5,  File: Outdoor_2024-04-01 15_53_39.csv
Label: 1.0, Prediction: [0.014 0.986], Description: Frei, People: viel,  File: Outdoor_2024-03-30 21_25_43.csv
Label: 1.0, Prediction: [0.01531535 0.9846846